In [ ]:
import pandas as pd
import numpy as np
import nltk

In [15]:
from sklearn.metrics import accuracy_score

In [16]:
np.random.seed(500)

In [17]:
df = pd.read_csv("../3_Preprocessing/dataset_lemmatized.csv")
print(df.shape)
df.head()

(39566, 7)


,index,name,link,lang_eng,valid_dir,value,type
0,0,Australian National University,http://www.anu.edu.au/,1,1,anu newsroom anu study anu research innovation...,Education
1,1,Monash University,http://www.monash.edu/,1,1,monash university one top university australia...,Education
2,2,University of Queensland,http://www.uq.edu.au/,1,1,university queensland university queensland ch...,Education
3,3,Macquarie University,http://mq.edu.au/,1,1,macquarie university tertiary education curren...,Education
4,4,UNSW Sydney,https://www.unsw.edu.au/,1,1,unsw sydney one best university australia unsw...,Education


In [18]:
# df = df[df["type"] != "Other"]
df = df[df["type"] != "Nonprofit"]
print(df.shape)
print(df['type'].unique())

(36430, 7)
['Education' 'Nonprofit' 'Facility' 'Government' 'Company' 'Healthcare'
 'Archive']


In [19]:
nltk.download("wordnet")

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Martin\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [20]:
# from nltk.stem import WordNetLemmatizer
# lemmatizer = WordNetLemmatizer()
#
# df['value'] = df['value'].apply(lambda x: " ".join([lemmatizer.lemmatize(w) for w in x.split()]))
# df.head()

In [21]:
from sklearn import model_selection
Train_X, Test_X, Train_Y, Test_Y = model_selection.train_test_split(df['value'], df['type'],test_size=0.3)

In [22]:
from sklearn.preprocessing import LabelEncoder
Encoder = LabelEncoder()
Train_Y = Encoder.fit_transform(Train_Y)
Test_Y = Encoder.fit_transform(Test_Y)

In [23]:
from sklearn.feature_extraction.text import TfidfVectorizer
Tfidf_vect = TfidfVectorizer(max_features=5000)
Tfidf_vect.fit(df['value'])
Train_X_Tfidf = Tfidf_vect.transform(Train_X)
Test_X_Tfidf = Tfidf_vect.transform(Test_X)

In [24]:
print(Tfidf_vect.vocabulary_)

{'newsroom': 3017, 'study': 4332, 'research': 3792, 'innovation': 2308, 'giving': 1901, 'alumnus': 176, 'current': 1101, 'student': 4330, 'strategic': 4308, 'initiative': 2297, 'experience': 1596, 'safety': 3914, 'security': 3999, 'staff': 4256, 'service': 4047, 'library': 2586, 'job': 2436, 'directory': 1277, 'pathway': 3286, 'explore': 1604, 'support': 4375, 'better': 480, 'future': 1841, 'event': 1558, 'turn': 4647, 'news': 3015, 'web': 4862, 'map': 2721, 'university': 4701, 'one': 3134, 'top': 4554, 'australia': 367, 'equity': 1524, 'diversity': 1306, 'inclusion': 2237, 'framework': 1792, 'priority': 3518, 'area': 286, 'data': 1135, 'governance': 1930, 'choose': 787, 'online': 3136, 'learning': 2555, 'course': 1044, 'diploma': 1269, 'degree': 1172, 'freedom': 1803, 'speech': 4221, 'academic': 11, 'supporting': 4378, 'diverse': 1305, 'inclusive': 2238, 'culture': 1094, 'world': 4953, 'class': 815, 'education': 1406, 'integrity': 2347, 'respect': 3808, 'new': 3010, 'cardiac': 673, 'c

In [25]:
print(Train_X_Tfidf)

  (0, 4862)	0.6708011446658749
  (0, 4628)	0.19963738172514872
  (0, 4496)	0.12821398794114477
  (0, 4014)	0.09003886865150434
  (0, 3510)	0.04159042238851357
  (0, 3449)	0.2046246054218944
  (0, 3107)	0.19254717809750593
  (0, 3069)	0.13687470340479707
  (0, 2527)	0.07288293154898888
  (0, 1932)	0.10969452865252861
  (0, 1793)	0.102367240357513
  (0, 1712)	0.2124009614619126
  (0, 1529)	0.17963983266471864
  (0, 1485)	0.03536752159722239
  (0, 978)	0.048100789034574626
  (0, 730)	0.19121929757374834
  (0, 654)	0.49079217044199297
  (1, 4800)	0.15464833872159117
  (1, 4460)	0.12419291459408624
  (1, 4454)	0.18894259173797728
  (1, 4375)	0.12057150351038749
  (1, 4176)	0.13426916420472423
  (1, 4047)	0.09971243413778558
  (1, 3685)	0.28640693671285833
  (1, 3356)	0.29053826383834164
  :	:
  (25500, 3315)	0.048261846545618096
  (25500, 3015)	0.01270307327525691
  (25500, 2708)	0.0368293380246305
  (25500, 2436)	0.0429401203868124
  (25500, 2385)	0.049627961457332434
  (25500, 2282)	0.028

In [26]:
# fit the training dataset on the NB classifier
from sklearn import naive_bayes
Naive = naive_bayes.MultinomialNB()
Naive.fit(Train_X_Tfidf,Train_Y)

# predict the labels on validation dataset
predictions_NB = Naive.predict(Test_X_Tfidf)

# Use accuracy_score function to get the accuracy
print(f"Naive Bayes Accuracy Score -> {round(accuracy_score(predictions_NB, Test_Y)*100, 2)}%")

Naive Bayes Accuracy Score -> 78.19%


In [27]:
# Classifier - Algorithm - SVM
# fit the training dataset on the classifier
from sklearn import svm

SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
SVM.fit(Train_X_Tfidf,Train_Y)

# predict the labels on validation dataset
predictions_SVM = SVM.predict(Test_X_Tfidf)

# Use accuracy_score function to get the accuracy
print(f"SVM Accuracy Score -> {round(accuracy_score(predictions_SVM, Test_Y)*100, 2)}%",)

SVM Accuracy Score -> 80.52%


Vysledky accuracy su lepsie o 5% pri lemmatizovani oproti stemmingu